In [1]:
import numpy as np
import keras
from keras import backend as K
from tqdm import tqdm

/home/kullu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input
from keras.layers.core import Reshape,Dense,Dropout,Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Convolution2D,UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.datasets import mnist
from keras import initializers

In [3]:
(train_data,train_label),(test_data,test_label) = mnist.load_data()
train_data = train_data.astype('float32')/255
train_data = train_data.reshape(60000,784)

In [4]:
## generator

In [5]:
randomDim = 10
adam = keras.optimizers.Adam(lr=0.0001,beta_1=0.5)

generator = keras.Sequential()
generator.add(Dense(256,input_dim=randomDim,kernel_initializer=initializers.RandomNormal(stddev=0.02)))
generator.add(LeakyReLU(0.2))
generator.add(Dense(512))
generator.add(LeakyReLU(0.2))
generator.add(Dense(1024))
generator.add(LeakyReLU(0.2))
generator.add(Dense(784,activation='tanh'))
generator.compile(loss='binary_crossentropy',optimizer=adam)


In [6]:
## discriminator

In [7]:
discriminator = keras.Sequential()
discriminator.add(Dense(1024,input_dim=784,kernel_initializer=initializers.RandomNormal(stddev=0.02)))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(512))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(1,activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy',optimizer=adam)


In [8]:
## combine them both

In [9]:
discriminator.trainable = False
ganInput = Input(shape=(randomDim,))
x = generator(ganInput)
ganOutput = discriminator(x)
gan = Model(inputs=ganInput,outputs=ganOutput)
gan.compile(loss='binary_crossentropy',optimizer=adam)

In [10]:
dLoss = []
gLoss = []

In [11]:
def plotLoss(epoch):
    plt.figure(figsize=(10,8))
    plt.plot(dLoss,label='Discriminative Loss')
    plt.plot(gLoss,label='Generative Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('./images/gan_loss_epoch_%d.png' %epoch)

In [12]:
def plotGeneratedImages(epoch,examples=100,dim=(10,10),figsize=(10,10)):
    noise = np.random.normal(0,1,size=[examples,randomDim])
    generatedImages = generator.predict(noise)
    generatedImages = generatedImages.reshape(examples,28,28)
    
    plt.figure(figsize=figsize)
    for i in range(generatedImages.shape[0]):
        plt.subplot(dim[0],dim[1],i+1)
        plt.imshow(generatedImages[i],interpolation='nearest',cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('./images/gan_generated_image_epoch_%d.png'%epoch)
    

In [13]:
def saveModels(epoch):
    generator.save('./models/gan_generator_epoch_%d.h5'%epoch)
    discriminator.save('./models/gan_discriminator_epoch_%d.h5'%epoch)

In [14]:
def train(epochs=1,batchSize=128):
    batchCount = train_data.shape[0]/batchSize
    print('Epochs : ',epochs)
    print('Batch Size : ',batchSize)
    print('Batches per epoch : ',batchCount)
    
    for e in range(1,epochs+1):
        print('-'*15,'Epoch %d'%e,'-'*15)
        for _ in tqdm(range(int(batchCount))):
            noise = np.random.normal(0,1,size=[batchSize,randomDim])
            imageBatch = train_data[np.random.randint(0,train_data.shape[0],size=batchSize)]
            
            generatedImages = generator.predict(noise)
            # print np.shape(imageBatch), np.shape(generatedImages)
            X = np.concatenate([imageBatch, generatedImages])

            # Labels for generated and real data
            yDis = np.zeros(2*batchSize)
            # One-sided label smoothing
            yDis[:batchSize] = 0.9
            
            discriminator.trainable = True
            dloss = discriminator.train_on_batch(X, yDis)

            # Train generator
            noise = np.random.normal(0, 1, size=[batchSize, randomDim])
            yGen = np.ones(batchSize)
            discriminator.trainable = False
            gloss = gan.train_on_batch(noise, yGen)

        # Store loss of most recent batch from this epoch
        dLoss.append(dloss)
        gLoss.append(gloss)
        
        if e == 1 or e % 20 == 0:
            plotGeneratedImages(e)
            saveModels(e)

    # Plot losses from every epoch
    plotLoss(e)


In [ ]:
train(1000, 256)

  0%|          | 0/234 [00:00<?, ?it/s]

Epochs :  1000
Batch Size :  256
Batches per epoch :  234.375
--------------- Epoch 1 ---------------


  0%|          | 0/234 [00:00<?, ?it/s]

--------------- Epoch 2 ---------------


  0%|          | 0/234 [00:00<?, ?it/s]

...

  0%|          | 0/234 [00:00<?, ?it/s]

--------------- Epoch 353 ---------------


  0%|          | 0/234 [00:00<?, ?it/s]

--------------- Epoch 354 ---------------


 85%|████████▍ | 198/234 [00:45<00:08,  4.35it/s]